In [5]:
from meb.utils import utils
from meb.datasets import datasets
from meb.models import models

import os
from tqdm import tqdm
from typing import Callable, List, Tuple, Union
from functools import partial

import numpy as np
from numba import jit
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import f1_score, confusion_matrix
from skimage.transform import resize
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import torch
import torch.nn as nn
import torch.optim as optim
import timm



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
c = datasets.CrossDataset(resize=64, optical_flow=True)

In [3]:
df = c.data_frame
data = c.data

Test on Casme II with different sizes of datasets

In [8]:
from meb.core.train_eval import CrossDatasetValidation

In [10]:
class Config:
    action_units = ['AU1', 'AU2', 'AU4', 'AU7', 'AU12', 'AU14', 'AU15', 'AU17']
    device = torch.device("cuda:0")
    epochs = 200
    weight = torch.tensor((1 / df.loc[:, action_units].sum(0) * 100).tolist()).to(device)
    criterion = utils.MultiLabelBCELoss(weight=weight)
    evaluation = utils.MultiLabelF1Score(average="macro")
    # Optimizer
    learning_rate = 1e-4
    weight_decay = 1e-3
    optimizer = optim.Adam
    scheduler = None
    # Dataloader
    batch_size = 128
    train_transform = {"spatial": None, "temporal": None}
    test_transform = {"spatial": None, "temporal": None}
    model = models.SSSNet(output_size=len(action_units))

We only care about the results of casme2

In [15]:
#casme
idx = df["dataset"].isin(["casme", "casme2"])
CrossDatasetValidation(Config).validate_n_times(df[idx], data[idx])

100%|██████████| 5/5 [00:37<00:00,  7.43s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU7', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
86.7 & 79.9 & 88.8 & 46.7 & 53.8 & 60.0 & 79.1 & 73.0 & 71.0

Datasets:  ['casme', 'casme2', 'Average']
69.8 & 71.6 & 70.7


In [16]:
#casme + samm
idx = df["dataset"].isin(["casme", "casme2", "samm"])
CrossDatasetValidation(Config).validate_n_times(df[idx], data[idx])

100%|██████████| 5/5 [01:42<00:00, 20.51s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU7', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
82.5 & 84.4 & 86.0 & 62.8 & 65.7 & 66.8 & 75.0 & 74.0 & 74.6

Datasets:  ['casme', 'casme2', 'samm', 'Average']
70.2 & 78.5 & 70.0 & 72.9


In [20]:
#casme + samm + mmew
idx = df["dataset"].isin(["casme", "casme2", "samm", "mmew"])
CrossDatasetValidation(Config).validate_n_times(df[idx].reset_index(), data[idx])

100%|██████████| 5/5 [03:34<00:00, 42.97s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU7', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
87.2 & 83.1 & 89.1 & 69.2 & 68.4 & 66.0 & 65.6 & 75.0 & 75.5

Datasets:  ['casme', 'casme2', 'samm', 'mmew', 'Average']
70.0 & 78.0 & 74.9 & 73.6 & 74.1


In [21]:
#casme + samm + mmew + 4dmer
idx = df["dataset"].isin(["casme", "casme2", "samm", "mmew", "fourDmicro"])
CrossDatasetValidation(Config).validate_n_times(df[idx].reset_index(), data[idx])

100%|██████████| 5/5 [06:11<00:00, 74.38s/it]

AUS: ['AU1', 'AU2', 'AU4', 'AU7', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
86.8 & 84.4 & 90.6 & 72.2 & 69.5 & 63.0 & 69.6 & 75.5 & 76.4

Datasets:  ['casme', 'casme2', 'samm', 'fourDmicro', 'mmew', 'Average']
71.8 & 80.0 & 74.2 & 75.3 & 75.1 & 75.3


evaluation on casme2 with different sets of datasets
casme 71.6
+samm 78.5
+mmew 78.0
+4dmer 80.0